In [1]:
import torch
import numpy as np

import sys
from pathlib import Path

sys.path.append('/home/nikita/e2e-driving')

import time
from tqdm.notebook import tqdm
import torch.nn.functional as F
import math

from ibc import optimizers
from dataloading.nvidia import NvidiaValidationDataset
from metrics.metrics import calculate_open_loop_metrics
from pilotnet import PilotnetEBM


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    all_predictions = []
    inference_times = []
    progress_bar = tqdm(total=len(dataloader), smoothing=0)

    epoch_mae = 0.0
    ask_batch_timestamp = time.time()
    for i, (input, target, _) in enumerate(dataloader):
        recv_batch_timestap = time.time()

        inputs = input['image'].to(device)
        target = target.to(device, torch.float32)

        inference_start = time.perf_counter()
        preds = model(inputs)
        inference_end = time.perf_counter()

        inference_time = inference_end - inference_start
        inference_times.append(inference_time)

        mae = F.l1_loss(preds, target.view(-1, 1))
        mae_degrees = math.degrees(mae.item())
        epoch_mae += mae_degrees

        all_predictions.extend(preds.cpu().squeeze().numpy())

        progress_bar.update(1)
        progress_bar.set_description(f'MAE: {(epoch_mae / (i + 1)):.4f}')

        ask_batch_timestamp = time.time()

    avg_mae = epoch_mae / len(dataloader)
    result = np.array(all_predictions)
    return avg_mae, result


def calculate_metrics(fps, predictions, valid_loader):
    '''For steering angle only.'''

    frames_df = valid_loader.dataset.frames
    true_steering_angles = frames_df.steering_angle.to_numpy()
    metrics = calculate_open_loop_metrics(predictions, true_steering_angles, fps=fps)

    left_turns = frames_df["turn_signal"] == 0  # TODO: remove magic values
    left_metrics = calculate_open_loop_metrics(predictions[left_turns], true_steering_angles[left_turns], fps=fps)
    metrics["left_mae"] = left_metrics["mae"]

    straight = frames_df["turn_signal"] == 1
    straight_metrics = calculate_open_loop_metrics(predictions[straight], true_steering_angles[straight], fps=fps)
    metrics["straight_mae"] = straight_metrics["mae"]

    right_turns = frames_df["turn_signal"] == 2
    right_metrics = calculate_open_loop_metrics(predictions[right_turns], true_steering_angles[right_turns], fps=fps)
    metrics["right_mae"] = right_metrics["mae"]

    return metrics

In [3]:
dataset_path = '/data/Bolt/dataset-new-small/summer2021'
output_modality = 'steering_angle'
n_branches = 1
n_waypoints = 1
batch_size = 256
num_workers = 8

validset = NvidiaValidationDataset(Path(dataset_path), output_modality, n_branches, n_waypoints=1)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=False,
                                          num_workers=num_workers, pin_memory=True,
                                          persistent_workers=True)

/data/Bolt/dataset-new-small/summer2021/2021-05-28-15-19-48_e2e_sulaoja_20_30: lenght=10708, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-06-07-14-20-07_e2e_rec_ss6: lenght=25836, filtered=1
/data/Bolt/dataset-new-small/summer2021/2021-06-07-14-06-31_e2e_rec_ss6: lenght=3003, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-06-07-14-09-18_e2e_rec_ss6: lenght=4551, filtered=1
/data/Bolt/dataset-new-small/summer2021/2021-06-07-14-36-16_e2e_rec_ss6: lenght=25368, filtered=1
/data/Bolt/dataset-new-small/summer2021/2021-09-24-14-03-45_e2e_rec_ss11_backwards: lenght=25172, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-10-26-10-49-06_e2e_rec_ss20_elva: lenght=33045, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-10-26-11-08-59_e2e_rec_ss20_elva_back: lenght=33281, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-10-20-15-11-29_e2e_rec_vastse_ss13_17_back: lenght=26763, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-10-11-14-50-59_e2e_rec_va

In [7]:
pt_model_path = '/home/nikita/e2e-driving/models/20220618185552_steering-angle/last.pt'

def evaluate_model(**args):
  model = PilotnetEBM()
  model = optimizers.DFOptimizer(model, optimizers.DerivativeFreeConfig(**args))
  model.load_state_dict(torch.load(pt_model_path))
  model.to(device)
  mae, preds = evaluate(model, valid_loader)

  return mae, preds

In [5]:
inputs = iter(valid_loader).next()[0]['image'][0].unsqueeze(0).to(device)
print(inputs.shape)

torch.Size([1, 3, 68, 264])


In [9]:
import itertools
import json
import wandb

fps = 30

samples = [256, 1024, 4096]
iters = [1, 3, 10, 30]
noise_scale = [0.1, 0.33, 1., 3.]
noise_shrink = 0.5

run_hparams = list(itertools.product(samples, iters, noise_scale))

for idx, (samples, iters, noise_scale) in enumerate(run_hparams):
    # wandb.init(project="ibc-tuning", name=f'{samples}s{iters}it{noise_scale}ns', config={"samples": samples, "iters": iters, "noise_scale": noise_scale, "noise_shrink": 0.5, "model_path": pt_model_path})

    print(f'running experiment {idx+1}/{len(run_hparams)}. samples: {samples}, iters: {iters}, noise_scale: {noise_scale}')
    mae, preds = evaluate_model(inference_samples=samples, iters=iters, noise_scale=noise_scale, noise_shrink=0.5)
    metrics = calculate_metrics(fps, preds, valid_loader)
    # wandb.log(metrics)
    print('experiment metrics:')
    print(json.dumps(str(metrics), indent=2))
    print()

running experiment 1/48. samples: 256, iters: 1, noise_scale: 0.1


  0%|          | 0/869 [00:00<?, ?it/s]

KeyboardInterrupt: 